In [16]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
key = os.getenv("API_KEY_FOOTBALL")
pd.set_option('display.max_columns', None)

# Configuração de cabeçalho, chave e host


In [17]:
headers = {
    "x-rapidapi-host": "v3.football.api-sports.io",
    "x-rapidapi-key": key  # Substitua pela sua chave de API
}

In [3]:
liga = 'Camp_Brasileiro'
season = '2012'

In [4]:
params = {
    "league": 71,  # ID da liga que analisaremos (71 = Campeonato Brasileiro) (140 - La liga) (39 PremierLeague)
    "season": 2012  # Temporada
}

# Resultado de todos os jogos

In [18]:
url_jogos = "https://v3.football.api-sports.io/fixtures/"

# Requisição GET
response = requests.get(url_jogos, headers=headers, params=params)

#Salvando os dados json
dados = response.json()

df = pd.DataFrame(dados['response'])

#Deixando em um formato legivel

df_expanded = pd.concat([
    pd.json_normalize(df['fixture']),  # Expandir informações de 'fixture'
    pd.json_normalize(df['league']),  # Expandir informações de 'league'
    pd.json_normalize(df['teams']),   # Expandir informações de 'teams'
    pd.json_normalize(df['goals']),   # Expandir informações de 'goals'
    pd.json_normalize(df['score'])    # Expandir informações de 'score'
], axis=1)

#Formatando o nome das colunas

df_expanded = df_expanded[['id','referee','date','venue.name','home.name','home.id','away.name','away.id','home.winner','home','away','halftime.home','halftime.away']]
df_games = df_expanded
df_games.columns = ['fixture_id', 'id_league','referee', 'date', 'venue.name', 'home.name', 'home.id', 'away.name',
       'away.id', 'home.winner', 'home', 'away', 'halftime.home',
       'halftime.away']

#Salvando csv
df_games.to_csv(f'../Data/{liga}_{season}_Games.csv', index=False)

KeyError: 'fixture'

- Vou Precisar de todos os id_partida para localizar outros dados

In [6]:
fixture_ids = df_games['fixture_id'].tolist()

# Request de todos os dados dos TIMES durante uma id_partida

- Coletando os dados

In [7]:
#Request dos dados das partidas

# Criando uma lista vazia para armazenar as estatísticas
all_stats = []

#De onde vou puxar as estatisticas da partida
url_stats = "https://v3.football.api-sports.io/fixtures/statistics"

#Varrendo os id_partidas
for fixture_id in fixture_ids:
    # Parâmetros da requisição
    params = {"fixture": fixture_id}
    
    # Requisição GET
    response = requests.get(url_stats, headers=headers, params=params)
    stats_data = response.json().get("response", [])
    all_stats.append({
    "fixture_id": fixture_id,
    "stats": stats_data
        })

- Processando os dados e salvando csv

In [8]:
# Lista para armazenar os dados processados
processed_data = []

# Iterar sobre cada partida em all_stats
for match in all_stats:
    fixture_id = match['fixture_id']
    
    # Iterar sobre as estatísticas de cada time
    for team_stats in match['stats']:
        team_id = team_stats['team']['id']
        team_name = team_stats['team']['name']
        
        # Criar um dicionário para armazenar os dados do time
        team_data = {
            'fixture_id': fixture_id,
            'team_id': team_id,
            'team_name': team_name
        }
        
        # Adicionar as estatísticas ao dicionário
        for stat in team_stats['statistics']:
            stat_type = stat['type'].lower().replace(' ', '_')  # Padroniza o nome da coluna
            stat_value = stat['value']
            team_data[stat_type] = stat_value
        
        # Adicionar o dicionário à lista processada
        processed_data.append(team_data)

# Criar o DataFrame
df_statistics_games = pd.DataFrame(processed_data)

#Salvando csv
df_statistics_games.to_csv(f'../Data/{liga}_{season}_team_stats.csv', index=False)

# Eventos que acontecem durante o jogo:

- Gols    
- Assistências
- Cartões
- Acréscimo
- Substituições

# Request

In [9]:
#Eventos que ocorreram durante o jogo 
url_events = "https://v3.football.api-sports.io/fixtures/events"

# Criar uma lista para armazenar os eventos
all_events = []

# Loop para buscar eventos de cada partida
for fixture_id in fixture_ids:
    # Parâmetros da requisição
    params = {"fixture": fixture_id}
    
    # Requisição GET
    response = requests.get(url_events, headers=headers, params=params)
    
    if response.status_code == 200:
        events_data = response.json().get("response", [])
        
        # Adicionar os eventos ao array
        all_events.append({
            "fixture_id": fixture_id,
            "events": events_data
        })
    else:
        print(f"Erro ao buscar eventos para fixture_id {fixture_id}: {response.status_code}")

- Processando e salvando csv

In [10]:
# Lista para armazenar os eventos processados
rows = []

# Iterar sobre cada fixture e seus eventos
for fixture in all_events:
    fixture_id = fixture['fixture_id']
    for event in fixture['events']:
        # Criar uma nova linha com o fixture_id
        row = {'fixture_id': fixture_id}
        
        # Processar cada campo do evento
        for key, value in event.items():
            # Desestruturar dicionários aninhados (time, team, player, assist)
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    row[f"{key}_{sub_key}"] = sub_value
            else:
                row[key] = value
        
        rows.append(row)

# Criar o DataFrame
df = pd.DataFrame(rows)

# Definir uma ordem lógica para as colunas
column_order = [
    'fixture_id',
    'time_elapsed',
    'time_extra',
    'team_id',
    'team_name',
    'team_logo',
    'player_id',
    'player_name',
    'assist_id',
    'assist_name',
    'type',
    'detail',
    'comments'
]

# Reordenar as colunas
df = df[column_order]
df_events = df

#Salvando csv
df.to_csv(f'../Data/{liga}_{season}_events_stats.csv', index=False)

# Escalações dos times

# Request

In [11]:
url_lineups = "https://v3.football.api-sports.io/fixtures/lineups"

all_lineups = []

for fixture_id in fixture_ids:
    # Parâmetros da requisição
    params = {"fixture": fixture_id}
    
    # Requisição GET
    response = requests.get(url_lineups, headers=headers, params=params)
    
    if response.status_code == 200:
        lineup_data = response.json().get("response", [])
        all_lineups.append({
            "fixture_id": fixture_id,
            "lineups": lineup_data
        })
    else:
        print(f"Erro ao buscar lineups para fixture_id {fixture_id}: {response.status_code}")

- Processando e salvando csv

In [12]:
data = []

# Iterar sobre cada evento
for event in all_lineups:
    fixture_id = event['fixture_id']
    
    # Iterar sobre cada time no evento
    for lineup in event['lineups']:
        team_name = lineup['team']['name']
        formation = lineup['formation']
        coach = lineup['coach']['name']
        
        # Adicionar os titulares
        for player in lineup['startXI']:
            data.append({
                'fixture_id': fixture_id,
                'team': team_name,
                'formation': formation,
                'coach': coach,
                'player_id': player['player']['id'],
                'player_name': player['player']['name'],
                'number': player['player']['number'],
                'position': player['player']['pos'],
                'grid': player['player']['grid'],
                'status': 'starter'
            })
        
        # Adicionar os substitutos
        for player in lineup['substitutes']:
            data.append({
                'fixture_id': fixture_id,
                'team': team_name,
                'formation': formation,
                'coach': coach,
                'player_id': player['player']['id'],
                'player_name': player['player']['name'],
                'number': player['player']['number'],
                'position': player['player']['pos'],
                'grid': player['player']['grid'],
                'status': 'substitute'
            })
df_lines = pd.DataFrame(data)

# Salvando csv

df_lines.to_csv(f'../Data/{liga}_{season}_escalacao.csv', index=False)

# Estatísticas dos players durante os jogos

# Request

In [13]:
#Estatisticas dos players 

url_players_stats = "https://v3.football.api-sports.io/fixtures/players"


# Lista para armazenar as estatísticas dos jogadores
all_players_stats = []

# Loop para buscar as estatísticas de cada partida
for fixture_id in fixture_ids:
    # Parâmetros da requisição
    params = {"fixture": fixture_id}
    
    # Requisição GET
    response = requests.get(url_players_stats, headers=headers, params=params)
    
    if response.status_code == 200:
        players_data = response.json().get("response", [])
        for team_stats in players_data:
            team = team_stats.get("team", {})
            players = team_stats.get("players", [])
            for player in players:
                all_players_stats.append({
                    "fixture_id": fixture_id,
                    "team_id": team.get("id"),
                    "team_name": team.get("name"),
                    "player_id": player.get("player", {}).get("id"),
                    "player_name": player.get("player", {}).get("name"),
                    "player_statistics": player.get("statistics", [])
                })
    else:
        print(f"Erro ao buscar estatísticas de jogadores para fixture_id {fixture_id}: {response.status_code}")

- Processando e salvando csv

In [15]:
# Lista para armazenar os dados
data = []

# Iterar sobre cada jogador
for player in all_players_stats:
    fixture_id = player['fixture_id']
    team_id = player['team_id']
    team_name = player['team_name']
    player_id = player['player_id']
    player_name = player['player_name']
    
    # Extrair as estatísticas do jogador
    stats = player['player_statistics'][0]  # Acessa o primeiro (e único) item da lista de estatísticas
    
    # Adicionar os dados ao DataFrame
    data.append({
        'fixture_id': fixture_id,
        'team_id': team_id,
        'team_name': team_name,
        'player_id': player_id,
        'player_name': player_name,
        'minutes': stats['games']['minutes'],
        'number': stats['games']['number'],
        'position': stats['games']['position'],
        'rating': stats['games']['rating'],
        'captain': stats['games']['captain'],
        'substitute': stats['games']['substitute'],
        'offsides': stats['offsides'],
        'shots_total': stats['shots']['total'],
        'shots_on': stats['shots']['on'],
        'goals_total': stats['goals']['total'],
        'goals_conceded': stats['goals']['conceded'],
        'assists': stats['goals']['assists'],
        'saves': stats['goals']['saves'],
        'passes_total': stats['passes']['total'],
        'passes_key': stats['passes']['key'],
        'passes_accuracy': stats['passes']['accuracy'],
        'tackles_total': stats['tackles']['total'],
        'tackles_blocks': stats['tackles']['blocks'],
        'tackles_interceptions': stats['tackles']['interceptions'],
        'duels_total': stats['duels']['total'],
        'duels_won': stats['duels']['won'],
        'dribbles_attempts': stats['dribbles']['attempts'],
        'dribbles_success': stats['dribbles']['success'],
        'dribbles_past': stats['dribbles']['past'],
        'fouls_drawn': stats['fouls']['drawn'],
        'fouls_committed': stats['fouls']['committed'],
        'cards_yellow': stats['cards']['yellow'],
        'cards_red': stats['cards']['red'],
        'penalty_won': stats['penalty']['won'],
        'penalty_committed': stats['penalty']['commited'],
        'penalty_scored': stats['penalty']['scored'],
        'penalty_missed': stats['penalty']['missed'],
        'penalty_saved': stats['penalty']['saved']
    })

    #Pandas

    df_players_stats = pd.DataFrame(data)

# Salvando csv

df_players_stats.to_csv(f'../Data/{liga}_{season}_players_stats.csv', index=False)

NameError: name 'df_players_stats' is not defined